<a href="https://colab.research.google.com/github/Visels/ResumeRanker/blob/main/ResumeParserNER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#RESUME PARSING USING NER

##Installing necessary libraries

In [2]:
!pip install spacy_transformers
!pip install -U spacy
!pip install PyMuPDF

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##Importing necessary files

In [3]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json
import pickle
import locale

In [4]:
spacy.__version__

'3.5.2'

In [5]:
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!git clone https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git

Cloning into 'CV-Parsing-using-Spacy-3'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 82 (delta 16), reused 5 (delta 0), pack-reused 0
Unpacking objects: 100% (82/82), 5.33 MiB | 5.76 MiB/s, done.


In [6]:
# cv_data = pickle.load(open('/content/CV-Parsing-using-Spacy-3/data/training/train_data.json', 'rb'))
# cv_data = pickle.load("/content/CV-Parsing-using-Spacy-3/data/training/train_data.pkl")



# cv_data = pickle.load(open('/content/CV-Parsing-using-Spacy-3/data/training/train_data.pkl', 'rb'))
# cv_data = json.load(open('/content/CV-Parsing-using-Spacy-3/data/training/train_data.json'))
cv_data = json.load(open('/content/train_data.json'))

In [ ]:
import json

with open('train_data.json', 'r') as f:
    training_data = json.load(f)

for text,annot in training_data:

    for entity in annot['entities']:
        start, end, label = entity
        # Remove trailing white spaces from the entity text
        text = text[start:end].strip()
        example['text'] = example['text'][:start] + text + example['text'][end:]

with open('training_data_cleaned.json', 'w') as f:
    json.dump(training_data, f)

In [7]:
len(cv_data)

200

In [8]:
!python -m spacy init fill-config '/content/CV-Parsing-using-Spacy-3/data/training/base_config.cfg' config.cfg

2023-05-06 23:21:44.312744: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
cv_data[2]

In [ ]:
# def get_spacy_doc(file,data):
#   nlp = spacy.blank('en')
#   db = DocBin()

#   for text, annot in tqdm(data):
#     doc = nlp.make_doc(text)
#     annot = annot['entities']

#     ents = []
#     entity_indices = []

#     print(annot)
#     for start, end, label in annot:
#     # for start, end, label in annot['entities']:
#       skip_entity = False

#       for idx in range(start, end):
#         if idx  in entity_indices:
#           skip_entity= False
#           break
#         if skip_entity == True:
#           continue

#         entity_indices = entity_indices + list(range(start, end))

#         try:
#           span =  doc.char_span(start, end, label = label, alignment_mode= 'strict')
#         except:
#           continue
        
#         if span is None:
#           err_data = str([start, end])+ "  " + 'some error' + "\n"
#           # err_data = str([start, end]).join(text).encode('utf-8').strip()
#           file.write(err_data)

#         else:
#           ents.append(span)

#     try:
#       doc.ents = ents
#       db.add(doc)

#     except:
#       pass

#   return db

In [ ]:
# def get_spacy_doc(file,data):


#     nlp = spacy.blank('en')
#     db = DocBin()



#     for text, annot in tqdm(data):
#         doc = nlp.make_doc(text)
#         annot = annot['entities']

#         ents = []
#         entity_indices = []

        

    
#         for start, end, label in annot:
#           skip_entity = False
#           for idx in range(start, end):
#             if idx  in entity_indices:
#                 skip_entity= False
#                 break
#         if skip_entity == True:
#             continue
            
#         entity_indices = entity_indices + list(range(start, end))

#         try:
#             span =  doc.char_span(start, end, label = label, alignment_mode= 'strict')
#         except:
#             continue

        
          
#         if span is None:
#             err_data = str([start, end])+ "  " + 'some error' + "\n"
#             # err_data = str([start, end]).join(text).encode('utf-8').strip()
#             file.write(err_data)
#         else:
#             ents.append(span)

#         try:
#             doc.ents = ents
#             db.add(doc)

#         except:
#             pass
       

#     return db
  
    
#I was using this
        

In [9]:
def get_spacy_doc(file,data):
    nlp = spacy.blank('en')
    db = DocBin()


    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)
        annot = annot['entities']
        ents = []
        entity_indices = []

        for start,end, label in annot:
            skip_entity = False

            for idx in range(start, end):
                if idx in entity_indices:
                    skip_entity = True
                    break

            if skip_entity == True:
                continue

            entity_indices = entity_indices  + list(range(start, end))

            try:
                span = doc.char_span(start, end, label = label, alignment_mode = 'strict')
            except:
                continue
            
            if span is None:
                # err_data = str([start, end] + " "  + "\n")
                err_data = f"error occured at [{start}, {end}] : some error \n"
                file.write(err_data)
            if span is not None:
                ents.append(span)
    
    
        try:
            doc.ents = ents
            db.add(doc)
        except:
            pass

    return db



In [ ]:

import re
def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    # invalid_span_tokens = re.compile(r'\s')
    invalid_span_tokens = re.compile(r"^\s+|\s+$")

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])

    return cleaned_data

In [ ]:
import re

def clean_entity_spans(data: list) -> list:
  invalid_span_tokens = re.compile(r'\s')

  cleaned_data = []

  for content in data:
      name = content['documentName']
      text = content['document']
      userinput = content['user_input']

      valid_entities = []

      for annotate_content in content['annotation']:
          start = annotate_content['start']
          end = annotate_content['end']
          label = annotate_content['label']
          text1 = annotate_content['text']

          valid_start = start
          valid_end = end

          while valid_start < len(text) and invalid_span_tokens.match(
                  text[valid_start]):
              valid_start += 1
          while valid_end > 1 and invalid_span_tokens.match(
                  text[valid_end - 1]):
              valid_end -= 1
          
          valid_entities.append({'start': valid_start, 'end': valid_end, 'label': label, 'text': text1, 'propertiesList': [], 'commentsList': []})
      cleaned_data.append({'documentName': name, 'document':text, 'annotation': valid_entities, 'user_input': userinput})

  return cleaned_data

In [ ]:
# cv_data = trim_entity_spans(cv_data)
cv_data

In [11]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(cv_data, test_size = 0.3)

print("Training data length:", len(train))
print("Test data length:", len(test))

Training data length: 140
Test data length: 60


In [21]:
file = open('error.txt', 'w')

db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

file.close()

100%|██████████| 60/60 [00:00<00:00, 64.28it/s]


In [ ]:
# viewing the tokens
db.tokens.count

<function list.count(value, /)>

In [ ]:
!python -m spacy train /content/config.cfg  --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0
# !python -m spacy train /content/config.cfg  --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy 

2023-05-06 23:58:05.080160: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-05-06 23:58:14,990] [INFO] Set up nlp object from config
[2023-05-06 23:58:15,006] [INFO] Pipeline: ['transformer', 'ner']
[2023-05-06 23:58:15,009] [INFO] Created vocabulary
[2023-05-06 23:58:15,011] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected 

In [ ]:
!python -m spacy debug data /content/config.cfg --paths.train /content/train_data.spacy


2023-05-06 21:09:49.394404: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

============================ Data file validation ============================
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
✔ Pipeline can be initialized with data
/usr/local/lib/python3.10/dist-packages/spacy/trainin

##Testing the Model

In [19]:
nlp = spacy.load('/content/output/model-last')
# nlp = spacy.load('/content/CV-Parsing-using-Spacy-3/nlp_model')

doc = nlp('Alice Clark  AI / Machine Learning Delhi, India Email me on Indeed • 20+ years of experience in data handling, design, and development• Data Warehouse: Data analysis, star/snow flake scema data modelling and design specific to data warehousing and business intelligence• Database: Experience in database designing, scalability, back-up and recovery, writing andoptimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes.Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure, Stream Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake analytics(U-SQL)Willing to relocate anywhereWORK EXPERIENCESoftware EngineerMicrosoft – Bangalore, KarnatakaJanuary 2000 to Present1. Microsoft Rewards Live dashboards:Description: - Microsoft rewards is loyalty program that rewards Users for browsing and shoppingonline. Microsoft Rewards members can earn points when searching with Bing, browsing withMicrosoft Edge and making purchases at the Xbox Store, the Windows Store and the MicrosoftStore. Plus, user can pick up bonus points for taking daily quizzes and tours on the Microsoftrewards website. Rewards live dashboards gives a live picture of usage world-wide and bymarkets like US, Canada, Australia, new user registration count, top/bottom performing reward offers, orders stats and weekly trends of user activities, orders and new user registrations. the PBI tiles gets refreshed in different frequencies starting from 5 seconds to 30 minutes.Technology/Tools used EDUCATION Indian Institute of Technology – Mumbai 2001 SKILLS Machine Learning, Natural Language Processing, and Big Data Handlin')

for ent in doc.ents:
  print(ent.text, '------>', ent.label_)

Microsoft ------> Companies worked at
Microsoft ------> Companies worked at
Microsoft ------> Companies worked at
Microsoft ------> Companies worked at
Indian Institute of Technology ------> College Name
Machine Learning, Natural Language Processing, and Big Data Handlin ------> Skills


In [17]:
doc = nlp('my name is Elvis Kipchumba and I worked at Microsoft I am good at data science and electrical engineering, I can code, program and do many other things')
print(doc)
for ent in doc.ents:
  print(ent.text, '------>', ent.label)

my name is Elvis Kipchumba and I worked at Microsoft I am good at data science and electrical engineering, I can code, program and do many other things
Microsoft ------> 4041747777783172106
